# Online Retail Dataset: Data Preparation
In this notebook, I'll prepare the dataset for analysis.
## Imports

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

## Read dataset

In [2]:
# Path to dataset
file_path = Path.cwd().parents[1] / "data" / "online_retail.xlsx"
assert file_path.exists(), f"file doesn't exist: {file_path}"

In [3]:
# Columns I'll actually use
cols = ["InvoiceNo", "InvoiceDate", "CustomerID", "Quantity", "UnitPrice"]

df = pd.read_excel(
    file_path,
    usecols=cols,
    dtype={
        "InvoiceNo": object,
        "CustomerID": object,
        "Quantity": np.int_,
        "UnitPrice": np.float_,
    },
    parse_dates=["InvoiceDate"],
).loc[:, cols]
df.head(10)

,InvoiceNo,InvoiceDate,CustomerID,Quantity,UnitPrice
0,536365,2010-12-01 08:26:00,17850,6,2.55
1,536365,2010-12-01 08:26:00,17850,6,3.39
2,536365,2010-12-01 08:26:00,17850,8,2.75
3,536365,2010-12-01 08:26:00,17850,6,3.39
4,536365,2010-12-01 08:26:00,17850,6,3.39
5,536365,2010-12-01 08:26:00,17850,2,7.65
6,536365,2010-12-01 08:26:00,17850,6,4.25
7,536366,2010-12-01 08:28:00,17850,6,1.85
8,536366,2010-12-01 08:28:00,17850,6,1.85
9,536367,2010-12-01 08:34:00,13047,32,1.69


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   InvoiceDate  541909 non-null  datetime64[ns]
 2   CustomerID   406829 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   UnitPrice    541909 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 20.7+ MB


## Data cleaning

In [5]:
# Number of missing values
df.isna().sum()

InvoiceNo           0
InvoiceDate         0
CustomerID     135080
Quantity            0
UnitPrice           0
dtype: int64

The only column that has missing values is `CustomerID`. But we really need
to know who bought what. Then rows with missing `CustomerID` have to go.

In [6]:
df = df[df.CustomerID.notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   InvoiceDate  406829 non-null  datetime64[ns]
 2   CustomerID   406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   UnitPrice    406829 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 18.6+ MB


`CustomerID` is supposed to be a 5-digit integer number. Let's confirm that
this is the case:

In [7]:
ids = df["CustomerID"].astype(str)
assert ids.str.len().eq(5).all()
assert ids.str.isdigit().all()
del ids

Looking for invalid quantities:

In [8]:
# `Quantity` should be strictly positive
# So this should equal 0
df.Quantity.le(0).sum()

8905

Not every row corresponds to a sale. When the invoice number starts with "C",
that transaction was canceled. That explains the observations with
non-positive quantities.

In [9]:
df["InvoiceNo"].astype(str).str.startswith("C").sum()

8905

In [10]:
tmp_df = df.loc[df["InvoiceNo"].astype(str).str.startswith("C"), ["InvoiceNo", "Quantity"]]
tmp_df.head(10)

,InvoiceNo,Quantity
141,C536379,-1
154,C536383,-1
235,C536391,-12
236,C536391,-24
237,C536391,-24
238,C536391,-24
239,C536391,-12
240,C536391,-12
241,C536391,-24
939,C536506,-6


In [11]:
assert tmp_df.Quantity.le(0).all()
del tmp_df

I chose to remove those rows:

In [12]:
df = df[df.Quantity > 0]

`InvoiceNo` is supposed to be a 6-digit integer number. Let's check that this
column is OK:

In [13]:
invoice_nums = df.InvoiceNo.astype(str)
assert invoice_nums.str.startswith("C").sum() == 0
assert invoice_nums.str.len().eq(6).all()
assert invoice_nums.str.isdigit().all()
del invoice_nums

Looking for invalid prices:

In [14]:
# Negative prices
df.UnitPrice.lt(0.0).sum()  # OK

0

In [15]:
# Products that cost nothing
df.UnitPrice.eq(0.0).sum()

40

I don't know how to explain such values. They should make no difference. Then
I chose to drop them:

In [16]:
df = df[df.UnitPrice > 0.0]

## Finish fixing dataset

In [17]:
# Use appropriate data types
df.InvoiceNo = df.InvoiceNo.astype("category")
assert not df.InvoiceNo.cat.ordered

df.CustomerID = df.CustomerID.astype("category")
assert not df.CustomerID.cat.ordered

In [18]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  category      
 1   InvoiceDate  397884 non-null  datetime64[ns]
 2   CustomerID   397884 non-null  category      
 3   Quantity     397884 non-null  int64         
 4   UnitPrice    397884 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1), int64(1)
memory usage: 11.4 MB


The only part of `InvoiceDate` that matters is the date. The following
command sets all the times to midnight:

In [19]:
df["InvoiceDate"] = df["InvoiceDate"].dt.normalize()
df["InvoiceDate"].head()

0   2010-12-01
1   2010-12-01
2   2010-12-01
3   2010-12-01
4   2010-12-01
Name: InvoiceDate, dtype: datetime64[ns]

For convenience, I'll collect the essential parts of the above code, and
create a function:

In [20]:
def get_clean_data(file_path: Path) -> pd.DataFrame:
    cols = ["InvoiceNo", "InvoiceDate", "CustomerID", "Quantity", "UnitPrice"]
    df = pd.read_excel(
        file_path,
        usecols=cols,
        dtype={
            "InvoiceNo": object,
            "CustomerID": object,
            "Quantity": np.int_,
            "UnitPrice": np.float_,
        },
        parse_dates=["InvoiceDate"],
    )
    return (
        df.loc[df.CustomerID.notna() & df.Quantity.gt(0) & df.UnitPrice.gt(0.0), cols]
        .assign(
            InvoiceNo=lambda x: x.InvoiceNo.astype("category"),
            InvoiceDate=lambda x: x.InvoiceDate.dt.normalize(),
            CustomerID=lambda x: x.CustomerID.astype("category"),
        )
        .reset_index(drop=True)
    )

In [21]:
# Quick check
df_func = get_clean_data(file_path)
assert_frame_equal(df_func, df)
del df_func

## Aggregate data
Before aggregating the data, I'll do some more consistency tests. Rows with
the same `InvoiceNo` must also have the same `InvoiceDate`. For a specific
value of `InvoiceNo`, this can be tested as follows:

In [22]:
assert df.loc[df["InvoiceNo"] == 536365, "InvoiceDate"].nunique() == 1

To test all values of `InvoiceNo`, one can do the following:

In [23]:
assert df.groupby(by="InvoiceNo", observed=True).InvoiceDate.nunique().eq(1).all()

Similarly, rows with the same `InvoiceNo` must also have the same
`CustomerID`. Checking if this is true:

In [24]:
# Single value
assert df.loc[df["InvoiceNo"] == 536365, "CustomerID"].nunique() == 1

In [25]:
# All values
assert df.groupby(by="InvoiceNo", observed=True).CustomerID.nunique().eq(1).all()

Everything is OK. Then I'll compute the total amount spent for each
`InvoiceNo`.

In [26]:
df_total = (
    df.assign(Price=lambda x: x.Quantity * x.UnitPrice)
    .groupby(by="InvoiceNo", observed=True)
    .agg({"InvoiceDate": "first", "CustomerID": "first", "Price": "sum"})
    .rename(columns={"Price": "TotalPrice"})
    .reset_index()
)
df_total.head()

,InvoiceNo,InvoiceDate,CustomerID,TotalPrice
0,536365,2010-12-01,17850,139.12
1,536366,2010-12-01,17850,22.20
2,536367,2010-12-01,13047,278.73
3,536368,2010-12-01,13047,70.05
4,536369,2010-12-01,13047,17.85


In [27]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18532 entries, 0 to 18531
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    18532 non-null  category      
 1   InvoiceDate  18532 non-null  datetime64[ns]
 2   CustomerID   18532 non-null  category      
 3   TotalPrice   18532 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1)
memory usage: 1.2 MB


I'll use this code in other notebooks. For this reason, I'll define a
function that returns the aggregated data. This function doesn't do much, but
it is convenient.

In [28]:
def compute_total_price(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df.assign(Price=lambda x: x.Quantity * x.UnitPrice)
        .groupby(by="InvoiceNo", observed=True)
        .agg({"InvoiceDate": "first", "CustomerID": "first", "Price": "sum"})
        .rename(columns={"Price": "TotalPrice"})
        .reset_index()
    )

In [29]:
def get_aggregated_data(file_path: Path) -> pd.DataFrame:
    return get_clean_data(file_path).pipe(compute_total_price)

In [30]:
# Quick check
df_func = get_aggregated_data(file_path)
assert_frame_equal(df_func, df_total)
del df_func

## Save prepared data
Clearly, I've ended up with a much smaller dataset than the original. To
avoid having to repeat the above steps, I'll save the new `DataFrame` to a
CSV file.

In [31]:
# Path to output CSV
out_file = file_path.with_suffix(".csv")

df_total.to_csv(out_file, index=False)